In [ ]:
conda init

In [ ]:
#2, Imports requires elements

import pandas as pd
import plotly
import numpy as np           # For math and missing values
import plotly.express as px  # For charts
import re                    # For searching text
from pathlib import Path     # For safe file paths
print("Setup complete! pandas:", pd.__version__)


In [ ]:
#3, Reads Excel Path, For required state file make sure to copy and paste the file name

RAW = Path("../data_raw")
CLEAN = Path("../data_clean"); CLEAN.mkdir(parents=True, exist_ok=True)
INS   = Path("../data_insights"); INS.mkdir(parents=True, exist_ok=True)

EXCEL_PATH = RAW / "GMU-Datastorm_Mississippi_Nov-2024.xlsx" #File name here
print("File location:", EXCEL_PATH.resolve())


In [ ]:
#4, Reads sheet name, sheet names need to be uniform and have 'XX_penalty' not 'XX_fee' or other improper naming

xls = pd.ExcelFile(EXCEL_PATH)
xls.sheet_names


In [ ]:
#5, Reads sheet name, If error occurs check excel file to see if the sheet name is 'XX_penalty'

pen = pd.read_excel(xls, sheet_name="az_penalty") #Change abreviated state name
pen.head(10)


In [ ]:
#6, Cleans the excel sheet

# 1. Clean column names (lowercase, underscores)
pen.columns = pen.columns.str.strip().str.lower().str.replace(" ", "_")

# 2. Turn text like "N/A" into proper missing values
def normalize_na(s):
    if isinstance(s, str):
        s = s.strip().lower()
        if s in {"n/a", "na", "none", ""}:
            return np.nan
    return s

for col in ["estimated_amount","penalty_other","penalty_type","statute_name"]:
    pen[col] = pen[col].apply(normalize_na)

# 3. Convert number columns to numeric
for c in ["year","penalty_amt_min","penalty_amt_max","penalty_amt_fixed"]:
    pen[c] = pd.to_numeric(pen[c], errors="coerce")

# 4. Drop rows missing key info
pen = pen.dropna(subset=["statute_id","year"]).copy()

pen.head()


In [ ]:
#7 New Statutes Visualization, if error occurs, ensure sheet name matches 'XX_penalty'
import pandas as pd
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from pathlib import Path

# --- Load dataset ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\GMU-Datastorm_Mississippi_Nov-2024.xlsx") #Change for state
df = pd.read_excel(file_path, sheet_name="az_penalty") #Change for state

# --- Clean year column ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df.dropna(subset=["year"])

# --- Aggregate: count statutes per year ---
statutes_per_year = (
    df.groupby("year")
    .size()
    .reset_index(name="statute_count")
)

# --- Plot ---
plt.figure(figsize=(8,5))
plt.bar(statutes_per_year["year"], statutes_per_year["statute_count"])
plt.title("New Statutes Introduced per Year")
plt.xlabel("Year")
plt.ylabel("Count")
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\az_visuals")
save_folder.mkdir(parents=True, exist_ok=True)  # create folder if it doesn't exist
save_path = save_folder / "01_new_statutes.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()

In [ ]:
#8, STATUTES CHANGED PER YEAR visualizations
import os
os.environ.pop("MPLBACKEND", None)  # Fix Matplotlib backend issue before importing
import matplotlib
matplotlib.use("TkAgg")

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# --- Load dataset ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\GMU-Datastorm_Mississippi_Nov-2024.xlsx")
df = pd.read_excel(file_path, sheet_name="az_penalty")

# --- Clean data ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["statute_id"] = df["statute_id"].astype(str)

# --- Filter out repealed rows and missing years ---
mask_repealed = df["penalty_type"].astype(str).str.contains("repeal", case=False, na=False)
df_filtered = df.loc[~mask_repealed & df["year"].notna()]

# --- Count unique statutes that had changes per year ---
statute_changes_per_year = (
    df_filtered.groupby("year")["statute_id"]
    .nunique()
    .reset_index()
    .rename(columns={"statute_id": "unique_statutes_changed"})
)

# --- Save output CSV ---
output_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_output\statutes_changed_per_year_cleanaz.csv")
output_path.parent.mkdir(parents=True, exist_ok=True)
statute_changes_per_year.to_csv(output_path, index=False)

# --- Plot ---
plt.figure(figsize=(8,5))
plt.bar(statute_changes_per_year["year"], statute_changes_per_year["unique_statutes_changed"])
plt.title("Number of Statutes with Changes per Year (Excluding Repealed)")
plt.xlabel("Year")
plt.ylabel("Number of Unique Statutes")
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\az_visuals")
save_folder.mkdir(parents=True, exist_ok=True)  # create folder if it doesn't exist
save_path = save_folder / "02_statutes_changed.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()


In [ ]:
#9, 10 PENALTIES OVER TIME GRAPH
import os
os.environ.pop("MPLBACKEND", None)
import matplotlib
matplotlib.use("TkAgg")

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import textwrap

# --- Load data ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\GMU-Datastorm_Mississippi_Nov-2024.xlsx")
df = pd.read_excel(file_path, sheet_name="az_penalty")

# --- Clean columns ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["penalty_type"] = df["penalty_type"].astype(str)
df["statute_id"] = df["statute_id"].astype(str)

# --- Filter to valid years ---
df = df.loc[df["year"].notna()]

# --- Identify top 10 most frequent penalty types overall ---
top_penalty_types = df["penalty_type"].value_counts().head(10).index
df_top = df[df["penalty_type"].isin(top_penalty_types)]

# --- Pivot: unique statutes per year per penalty type ---
pivot = (
    df_top.pivot_table(
        index="year",
        columns="penalty_type",
        values="statute_id",
        aggfunc="nunique"
    )
    .fillna(0)
)

# --- Wrap long legend labels ---
wrapped_cols = ['\n'.join(textwrap.wrap(col, 30)) for col in pivot.columns]

# --- Clear any auto-figures ---
plt.close('all')

# --- Plot ---
fig, ax = plt.subplots(figsize=(10, 6))
pivot.plot(kind="line", linewidth=2, ax=ax)

ax.set_title("Top 10 Penalty Types Over Time")
ax.set_xlabel("Year")
ax.set_ylabel("Unique Statutes (Non-Repealed)")

# Use wrapped labels in the legend
ax.legend(
    wrapped_cols,
    title="Penalty Type",
    bbox_to_anchor=(1.05, 1),
    loc="upper left"
)

ax.grid(True, linestyle="--", alpha=0.6)

# Adjust margins to fit legend
plt.subplots_adjust(left=0.1, right=0.8)
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\az_visuals")
save_folder.mkdir(parents=True, exist_ok=True)
save_path = save_folder / "03_10_penalties.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()



In [ ]:
#10, AVERAGE PENALTY OVER TIME GRAPH
import os
os.environ.pop("MPLBACKEND", None)  
import matplotlib
matplotlib.use("TkAgg") 

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# --- Load dataset ---
file_path = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\GMU-Datastorm_Mississippi_Nov-2024.xlsx")
df = pd.read_excel(file_path, sheet_name="az_penalty")

# --- Clean numeric columns ---
df["year"] = pd.to_numeric(df["year"], errors="coerce")
for col in ["penalty_amt_min", "penalty_amt_max", "penalty_amt_fixed", "estimated_amount"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- Compute effective penalty (take whichever is non-null and highest) ---
df["penalty_effective"] = df[["penalty_amt_fixed", "penalty_amt_max", "estimated_amount"]].max(axis=1)

# --- Filter for valid years and positive values ---
df_clean = df[(df["year"].notna()) & (df["penalty_effective"] > 0)]

# --- Average per year ---
avg_penalty = (
    df_clean.groupby("year")["penalty_effective"]
    .mean()
    .reset_index()
    .rename(columns={"penalty_effective": "avg_penalty"})
)

# --- Plot ---
plt.figure(figsize=(9, 5))
plt.plot(avg_penalty["year"], avg_penalty["avg_penalty"], marker="o", linewidth=2)
plt.title("Average Penalty Amount Over Time")
plt.xlabel("Year")
plt.ylabel("Average Penalty (USD)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()

# --- Save figure as PNG ---
save_folder = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\az_visuals")
save_folder.mkdir(parents=True, exist_ok=True)
save_path = save_folder / "04_average_penalties.png"
plt.savefig(save_path, dpi=300)

# --- Show plot ---
plt.show()


In [ ]:
#11, winsorization

def winsorize(series, lower=0.025, upper=0.975):
    return series.clip(lower=series.quantile(lower), upper=series.quantile(upper))

df["penalty_winsorized"] = winsorize(df["penalty_effective"])


In [ ]:
#12, discretionary text

import pandas as pd
import re

# Load your Excel file
pen = pd.read_excel(
    r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\data_raw\california_insightlegi_workingcopy_nov_2025.xlsx"
)

# ---- FIX: force numeric columns to numbers ----
AMOUNT_COLS = ["penalty_amt_min", "penalty_amt_max", "penalty_amt_fixed"]
for col in AMOUNT_COLS:
    pen[col] = pd.to_numeric(pen[col], errors="coerce")

# Define discretionary patterns
DISCRETION_PATTERNS = [
    r"judge", r"court", r"discretion", r"reasonable",
    r"to be determined", r"as determined",
    r"at (the )?court'?s discretion", r"by the court"
]
discretion_re = re.compile("|".join(DISCRETION_PATTERNS), flags=re.IGNORECASE)

# Function to check if text is discretionary
def is_discretionary(text):
    if not isinstance(text, str):
        return False
    return bool(discretion_re.search(text))

pen["is_discretionary_text"] = pen["penalty_other"].apply(is_discretionary)

# Mark numeric amounts
pen["has_min"]   = pen["penalty_amt_min"].notna() & (pen["penalty_amt_min"] > 0)
pen["has_max"]   = pen["penalty_amt_max"].notna() & (pen["penalty_amt_max"] > 0)
pen["has_fixed"] = pen["penalty_amt_fixed"].notna() & (pen["penalty_amt_fixed"] > 0)

pen["has_numeric_amount"] = pen[["has_min", "has_max", "has_fixed"]].any(axis=1)

# Classify amount type
def classify_amount_type(row):
    if row["has_fixed"]:
        return "fixed"
    if row["has_min"] or row["has_max"]:
        return "range"
    if row["is_discretionary_text"]:
        return "discretionary_text"
    return "none"

pen["amount_type"] = pen.apply(classify_amount_type, axis=1)

print(pen[["statute_id", "year", "penalty_other", "amount_type"]].head(10))

In [ ]:
#13, saving cleaned file

pen.to_csv(CLEAN / "penalties_az.csv", index=False)
print("Cleaned file saved to data_clean/")


In [ ]:
#14, fix year text
pen["year"] = pd.to_numeric(pen["year"], errors="coerce").astype("Int64")


In [ ]:
#15, Ensure year is numeric
pen["year"] = pd.to_numeric(pen["year"], errors="coerce").astype("Int64")

# Averages of penalty amounts per year
trend_amt = (
    pen.groupby("year", as_index=False)
       .agg(
           avg_min=("penalty_amt_min","mean"),
           avg_max=("penalty_amt_max","mean"),
           avg_fixed=("penalty_amt_fixed","mean"),
           n_statutes=("statute_id","nunique")
       )
       .sort_values("year")
)

# Share of each amount type per year
type_counts = (
    pen.groupby(["year","amount_type"])["statute_id"]
       .count()
       .reset_index(name="count")
)

year_totals = type_counts.groupby("year")["count"].sum().rename("year_total")
type_share = type_counts.merge(year_totals, on="year")
type_share["pct"] = (100 * type_share["count"] / type_share["year_total"]).round(1)

trend_amt.to_csv(INS / "penalty_trend_az.csv", index=False)
type_share.to_csv(INS / "penalty_amount_type_share_az.csv", index=False)

print("Trend and share tables saved.")



In [ ]:
#16, 2 graphs
# Ensure Kaleido is installed ---
try:
    import kaleido
except ImportError:
    print("Kaleido not found. Installing now...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "kaleido"])
    import kaleido

# --- Example data (replace with your own DataFrames) ---
# trend_amt = ...
# type_share = ...

# --- Create Line chart of average penalties ---
cols = ["avg_min","avg_max","avg_fixed"]
fig1 = px.line(trend_amt, x="year", y=cols, markers=True,
               title="Arizona – Average Penalty Levels Over Time")

# --- Create Area chart of amount types ---
comp_pivot = type_share.pivot(index="year", columns="amount_type", values="pct").fillna(0).reset_index()
fig2 = px.area(comp_pivot, x="year",
               y=[c for c in comp_pivot.columns if c != "year"],
               title="Arizona – Share of Amount Types by Year",
               groupnorm="fraction")
fig2.update_yaxes(ticksuffix="%")

# --- Output directory ---
output_dir = Path(r"C:\Users\antho\OneDrive\Documents\insightlegi-lite\Figures\AZ_visuals")
output_dir.mkdir(parents=True, exist_ok=True)

# --- Save figures ---
fig1.write_image(output_dir / "05_line_chart.png")
fig2.write_image(output_dir / "06_area_chart.png")

# --- Show figures in notebook / script ---
fig1.show()
fig2.show()




In [ ]:
missing = (100 * pen[["penalty_amt_min","penalty_amt_max","penalty_amt_fixed"]].isna().mean()).round(1)
print("Missing numeric fields (%):\n", missing)
print("\nAmount types overall:\n", pen["amount_type"].value_counts(dropna=False))
